In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw2.ipynb")

# Homework 2: Sampling Error and Bias
## Due Date: Thursday, Sept 9, 11:59PM

## Collaboration Policy

Data science is a collaborative activity. While you may talk with others about
the homework, we ask that you **write your solutions individually**. If you do
discuss the assignments with others please **include their names** below.

**Collaborators**: *list  collaborators here*

## Grading

Grading is broken down into autograded answers and free response. 

For autograded answers, the results of your code are compared to provided and/or hidden tests. **For autograded probability questions, the provided tests will only check that your answer is within a reasonable range.**

For free response, readers will evaluate how well you answered the question and/or fulfilled the requirements of the question.

For plots, make sure to be as descriptive as possible: include titles, axes labels, and units wherever applicable.

### Please make sure to save regularly; do not rely on autosave to save your work for you!

## Scoring Breakdown

|Question|Points|
|---|---|
|1.1|1|
|1.2|1|
|1.3|1|
|2.1|1|
|2.2|1|
|2.3|2|
|3.1|1|
|3.2|1|
|3.3|1|
|3.4|1|
|3.5|1|
|3.6|2|
|4.1|1|
|4.2|1|
|5|1|
|6.1|2|
|6.2|1|
|6.3|1|
|6.4|1|
|6.5|2|
|6.6|1|
|7.1|1|
|7.2|1|
|7.3|2|
|7.4|1|
|8.1|1|
|8.2|2|
|9|2|
|**Total**|35|


## Introduction

The outcome of the US presidential election in 2016 took many people and many pollsters by surprise. In this assignment we will carry out a simulation study / post mortem in an attempt to understand what happened. Doing such an analysis is especially important even though the 2020 federal elections have already occurred.

In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')

# Default plot configurations
%matplotlib inline
plt.rcParams['figure.figsize'] = (16,8)
plt.rcParams['figure.dpi'] = 150
sns.set()

from IPython.display import display, Latex, Markdown

## `SciPy` and `special` ##

Factorials and the *binomial coefficients* $\binom{n}{k} = \frac{n!}{k!(n-k)!}$ get large very quickly as $n$ gets large. One way to compute them is to use the `SciPy` module `special`. `SciPy` is a collection of Python-based software for math, probability, statistics, science, and engineering.

In [2]:
from scipy import special

Below are some examples of `special.factorial`:

In [3]:
special.factorial(5), special.factorial(range(1, 6))

Traditionally, subsets of $k$ individuals out of a population of $n$ individuals are called *combinations*, and so `special.comb(n, k)` evaluates to $\binom{n}{k}$. 

Note that we will always use the term *subsets* to mean un-ordered sets. We will use *permutations* in situations where we need to keep track of the order in which the elements appear.

Look at the code and output below carefully (including types) before starting Question 1.

In [4]:
special.comb(5, 3), special.factorial(5) / (special.factorial(3) * special.factorial(2))

In [5]:
special.comb(5, range(6))

In [6]:
special.comb(100, 50), special.comb(100, 50, exact=True)

### Question 1

Consider a population in which a proportion $p$ of individuals are called "successes" (or 1, if you prefer) and the remaining proportion are rudely called "failures" (or 0).

As we saw in lecture, if you draw a sample of size $n$ (where $n$ is some positive integer) at random with replacement from the population, then the number of successes is a random variable that follows the binomial distribution. The probability of drawing $k$ successes and $n-k$ failures is $\binom{n}{k}p^k(1-p)^{n-k}$, for $0 \leq k \leq n$. To reduce writing, we will shorten "$k$ successes and $n-k$ failures" to "$k$ successes". In other words, "$k$ successes" means "exactly $k$ successes".

To formalize notation: if $X$ is the number of successes we draw, then $P(X = k) = \binom{n}{k}p^k(1-p)^{n-k}$.

### Question 1a) ###
Suppose you sample 100 times at random with replacement from a population in which 26% of the individuals are successes. Write a Python expression that evaluates to the chance that the sample has 20 successes. 

**Computational note:** Don't import any other libraries; just use the ones already imported and plug into the formula above. It's far from the best way numerically, but it is fine for the numbers involved in this Homework. 

<!--
BEGIN QUESTION
name: q1a
manual: false
points: 1
-->

In [7]:
prob_1a = ...
prob_1a

In [ ]:
grader.check("q1a")

### Question 1b) ###
Complete the cell with a Python expression that evaluates to an array whose elements are the chances of $k$ successes for $k = 0, 1, 2, \ldots, 100$. That is, `all_probs[k]` should contain $P(X = k)$, where $X$ is the random variable describing the number of successes drawn from the scenario outlined above.

<!--
BEGIN QUESTION
name: q1b
points: 1
-->

In [10]:
k = np.arange(101)
all_probs = ...
np.round(all_probs, 4), sum(all_probs)

In [ ]:
grader.check("q1b")

### Question 1c) ###


Complete the cell with an expression that evaluates to the chance that the number of successes in the sample is in the interval $26 \pm 10$ (inclusive on both sides). In other words, determine $P(16 \leq X \leq 36)$. **Hint: You might want to apply your Python indexing knowledge here.**

**Note: Please assign your answer to `prob_1c`. Because we use an autograder, please make sure your answer is a proportion between 0 and 1, not a percent between 0 and 100.**

<!--
BEGIN QUESTION
name: q1c
manual: false
points: 1
-->

In [13]:
prob_1c = ...

In [ ]:
grader.check("q1c")

### Question 2

Consider (once again) a sample of size $n$ drawn at random with replacement from a population in which a proportion $p$ of the individuals are called successes.

Let $S$ be the random variable that denotes the number of successes in our sample. (As stated above, $S$ follows the binomial distribution.) Then, the probability that the number of successes in our sample is **at most** $s$ (where $0 \leq s \leq n$) is

$$P(S \leq s) = P(S = 0) + P(S = 1) + ... + P(S = s) = \sum_{k=0}^s \binom{n}{k}p^k(1-p)^{n-k}$$

We obtain this by summing the probability that the number of successes is exactly $k$, for each value of $k = 0, 1, 2, ..., s$.

### Question 2a) ###

Please fill in the function `prob_at_most` which takes $n$, $p$, and $s$ and returns $P(S \le s)$ as defined above. If the inputs are invalid: for instance, if $p > 1$ OR $s > n$ then return 0. 

It might help to refer to the calculation in Question 1b.

<!--
BEGIN QUESTION
name: q2a
manual: false
points: 1
-->

In [16]:
def prob_at_most(n, p, s):
    """ 
    returns the probability of S <= s
    Input n: sample size; p : proportion; s: number of successes at most"""
    ...

In [ ]:
grader.check("q2a")

### Question 2b) ###
In an election, supporters of Candidate C are in a minority. Only 45% of the voters in the population favor the candidate.

Suppose a survey organization takes a sample of 200 voters at random with replacement from this population. Use `prob_at_most` to write an expression that evaluates to the chance that a majority (more than half) of the sampled voters favor Candidate C.

<!--
BEGIN QUESTION
name: q2b
manual: false
points: 1
-->

In [22]:
p_majority = ...
p_majority

In [ ]:
grader.check("q2b")

### Question 2c) ###
Suppose each of five survey organizations takes a sample of voters at random with replacement from the population of voters in Part **b**, independently of the samples drawn by the other organizations. 

- Three of the organizations use a sample size of 200
- One organization uses a sample size of 300
- One organization uses a sample size of 400

Write an expression that evaluates to the chance that in at least one of the five samples the majority of voters favor Candidate C. You can use any quantity or function defined earlier in this exercise.

<!--
BEGIN QUESTION
name: q2c
manual: false
points: 2
-->

In [25]:
prob_2c = ...
prob_2c

In [ ]:
grader.check("q2c")

### Question 3

In Pennsylvania, 6,165,478 people voted in the 2016 Presidential election.
Trump received 48.18% of the vote and Clinton recieved 47.46%.
This doesn't add up to 100% because other candidates received votes.
All together these other candidates received 100% - 48.18% - 47.46% = 4.36% of the vote.

The table below displays the counts and proportions.


|   Voted for   |  Trump|    Clinton|    Other|
|-----------|-----------|-----------|---------|
| Probability      |   0.4818   | 0.4746  |   0.0436 |
| Number of people | 2,970,733  | 2,926,441 | 268,304 |

### Question 3a) ###
Suppose we pick a simple random sample of 20 of the 6,165,478 Pennsylvania (PA) voters. 

In the sample, let $N_T$ be the number of Trump voters, $N_C$ the number of Clinton voters, and $N_O$ the number of "other" voters. Then $N_T$, $N_C$, and $N_O$ are random: they depend on how the sample comes out. In Data 8 we called such quantities "statistics".

Pick the correct option: $N_T + N_C + N_O$ is equal to

<ol style="list-style-type: lower-alpha;">
    <li>3</li>
    <li>20</li>
    <li>6,165,478</li>
    <li>a random quantity</li>
</ol>

*Note*: if your answer is (d), put  `ans_3a = 'd'`  for the purpose of grading. Passing the test **does NOT** mean you answer is correct. The test only checks if your answer is in the correct format.

<!--
BEGIN QUESTION
name: q3a
manual: false
points: 
    - 0
    - 1
-->

In [28]:
ans_3a = ...

In [ ]:
grader.check("q3a")

### Question 3b) ###
Pick the correct option.

A simple random sample of 20 PA voters is like a sample drawn at random with replacement, because

<ol style="list-style-type: lower-alpha;">
<li>that's the definition of "simple random sample"</li>
<li>there are only 3 categories of voters, which is small in comparison to 20</li>
<li>there are only 20 people in the sample, which is small in comparison to the total number of PA voters</li>
<li>all PA voters are equally likely to be selected</li>
</ol>

*Note*: if your answer is (d), put `ans_3b = 'd'`  for the purpose of grading. Passing the test **does NOT** mean you answer is correct. The test only checks if your answer is in the correct format.

<!--
BEGIN QUESTION
name: q3b
manual: false
points: 
    - 0
    - 1
-->

In [31]:
ans_3b = ...

In [ ]:
grader.check("q3b")

### Question 3c) ###

Let $t$, $c$, and $o$ be any three non-negative integers. For the sample defined in Part **a**, the probability that the sample contains $t$ Trump voters, $c$ Clinton voters and $o$ "other" voters is denoted by $P(N_T = t, N_C = c, N_O = o)$.

Define a function `prob_sample_counts` that takes any three non-negative integers $t$, $c$, and $o$, and returns $P(N_T = t, N_C = c, N_O = o)$. The probability is 0 for some choices of the arguments, and your function should return 0 in those cases.

Remember that Part **b** implies you can use results for sampling with replacement.

What should `prob_sample_counts(31, 8, 1)` evaluate to? Check that your function returns the right value (No extra code/cells required).

<!--
BEGIN QUESTION
name: q3c
manual: false
points: 1
-->

In [34]:
def prob_sample_counts(t, c, o):
    """
    Input:
    t - number of votes for Trump
    c - number of votes for Clinton
    o - number of votes for Other
    Return: The probability of getting a such sample
    """
    ...

In [ ]:
grader.check("q3c")

### Question 3d) ###

Check that your function determines a probability distribution by summing all the positive terms. You know what the sum should be. **Hint: You might need to iterate through certain values of c and t to compute the sum of all positive probabilities.**

<!--
BEGIN QUESTION
name: q3d
manual: false
points: 1
-->

In [37]:
prob_3d = ...
prob_3d

In [ ]:
grader.check("q3d")

### Question 3e) ###
Use `prob_sample_counts` to find the chance that the sample consists of 11 Trump voters, 8 Clinton voters, and 1 "other" voter.

<!--
BEGIN QUESTION
name: q3e
manual: false
points: 1
-->

In [40]:
prob_3e = ...
prob_3e

In [ ]:
grader.check("q3e")

### Question 3f) ###
Write an expression that evaluates the chance that the sample contains at least 7 Trump voters, at least 7 Clinton voters, and at least 1 "other" voter. **Hint: You might find 3d useful.**

<!--
BEGIN QUESTION
name: q3f
manual: false
points: 2
-->

In [43]:
prob_3f = ...
prob_3f

In [ ]:
grader.check("q3f")

## Election Polling

Political polling is a type of public opinion polling that can at best represent a snapshot of public opinion at the particular moment in time. Voter opinion shifts from week to week, even day to day, as candidates battle it out on the campaign field.

Polls usually start with a "horse-race" question, where respondents are asked whom they would vote for in a head-to-head race if the election were tomorrow: Candidate A or Candidate B. The survey begins with this question so that the respondent is not influenced by any of the other questions asked in the survey. Some of these other questions are asked to help assess how likely is it that the respondent will vote. Other questions are asked about age, education, and sex in order to adjust the findings if one group appears overly represented in the sample.

Pollsters typically use [random digit dialing](https://en.wikipedia.org/wiki/Random_digit_dialing) to contact people.

### Question 4

<!-- BEGIN QUESTION -->

#### Part 1 ####

If we're trying to predict the results of the Clinton vs. Trump presidential race, what is the population of interest? 

<!--
BEGIN QUESTION
name: q4a
manual: true
points: 1
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

#### Part 2 ####

What is the sampling frame?

<!--
BEGIN QUESTION
name: q4b
manual: true
points: 1
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



### How might the sampling frame differ from the population?

After the fact, many experts have studied the 2016 election results. For example, according to the American Association for Public Opinion Research (AAPOR), predictions made before the election were flawed for three key reasons:

1. voters changed their preferences a few days before the election
2. those sampled were not representative of the voting population, e.g., some said that there was an overrepresentation of college graduates in some poll samples 
3. voters kept their support for Trump to themselves (hidden from the pollsters)

In Questions 6 and 7, we will do two things:

+ Question 6: We will carry out a study of the sampling error when there is no bias. In other words, we will try to compute the chance that we get the election result wrong even if we collect our sample in a manner that is completely correct. In this case, any failure of our prediction is due entirely to random chance.
+ Question 7: We will carry out a study of the sampling error when there is bias of the second type from the list above. In other words, we will try to compute the chance that we get the election result wrong if we have a small systematic bias. In this case, any failure of our prediction is due to a combination of random chance and our bias.


<!-- BEGIN QUESTION -->

### Question 5

Why can't we assess the impact of the other two biases (voters changing preference and voters hiding their preference)? 

Note: You might find it easier to complete this question after you've completed the rest of the homework including the simulation study.

<!--
BEGIN QUESTION
name: q5
manual: true
points: 1
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



### How large was the sampling error?

In some states the race was very close, and it may have been simply sampling error, i.e., random chance that the majority of the voters chosen for the sample voted for Clinton.

A 2- or 3-point polling error in Trump’s favor (typical error historically) would likely be enough to tip the Electoral College to him.

One year after the 2016 election, Nate Silver wrote in
*The Media Has A Probability Problem* that
the "media’s demand for certainty -- and its lack of statistical rigor -- is a bad match for our complex world."
FiveThirtyEight forecasted that Clinton had about a 70 percent chance of winning.  

We will first carry out a simulation study to assess the impact of the sampling error on the predictions.


## The Electoral College

The US president is chosen by the Electoral College, not by the
popular vote. Each state is alotted a certain number of 
electoral college votes, as a function of their population.
Whomever wins in the state gets all of the electoral college votes for that state.

There are 538 electoral college votes (hence the name of the Nate Silver's site, FiveThirtyEight).

Pollsters correctly predicted the election outcome in 46 of the 50 states. 
For these 46 states Trump received 231 and Clinton received 232 electoral college votes.

The remaining 4 states accounted for a total of 75 votes, and 
whichever candidate received the majority of the electoral college votes in these states would win the election. 

These states were Florida, Michigan, Pennsylvania, and Wisconsin.

|State |Electoral College Votes|
| --- | --- |
|Florida | 29 |
|Michigan | 16 |
|Pennsylvania | 20 |
|Wisconsin | 10|

For Donald Trump to win the election, he had to win either:
* Florida + one (or more) other states
* Michigan, Pennsylvania, and Wisconsin


The electoral margins were very narrow in these four states, as seen below:


|State | Trump |   Clinton | Total Voters |
| --- | --- |  --- |  --- |
|Florida | 49.02 | 47.82 | 9,419,886  | 
|Michigan | 47.50 | 47.27  |  4,799,284|
|Pennsylvania | 48.18 | 47.46 |  6,165,478|
|Wisconsin | 47.22 | 46.45  |  2,976,150|

Those narrow electoral margins can make it hard to predict the outcome given the sample sizes that the polls used. 

---
## Simulation Study of the Sampling Error

Now that we know how people actually voted, we can carry
out a simulation study that imitates the polling.

Our ultimate goal in this problem is to understand the chance that we will incorrectly call the election for Hillary Clinton even if our sample was collected with absolutely no bias.

### Question 6

#### Part 1 

For your convenience, the results of the vote in the four pivotal states is repeated below:

|State | Trump |   Clinton | Total Voters |
| --- | --- |  --- |  --- |
|Florida | 49.02 | 47.82 | 9,419,886  | 
|Michigan | 47.50 | 47.27  |  4,799,284|
|Pennsylvania | 48.18 | 47.46 |  6,165,478|
|Wisconsin | 47.22 | 46.45  |  2,976,150|


Using the table above, write a function `draw_state_sample(N, state)` that returns a sample with replacement of N voters from the given state. Your result should be returned as a list, where the first element is the number of Trump votes, the second element is the number of Clinton votes, and the third is the number of Other votes. For example, `draw_state_sample(1500, "florida")` could return `[727, 692, 81]`. You may assume that the state name is given in all lower case.

You might find `np.random.multinomial` useful.

<!--
BEGIN QUESTION
name: q6a
points: 2
-->

In [46]:
def draw_state_sample(N, state):
    ...

In [ ]:
grader.check("q6a")

#### Part 2

Now, create a function `trump_advantage` that takes in a sample of votes (like the one returned by `draw_state_sample`) and returns the difference in the proportion of votes between Trump and Clinton. For example `trump_advantage([100, 60, 40])` would return `0.2`, since Trump had 50% of the votes in this sample and Clinton had 30%.

<!--
BEGIN QUESTION
name: q6b
points: 1
-->

In [49]:
def trump_advantage(voter_sample):
    ...

In [ ]:
grader.check("q6b")

#### Part 3

Simulate Trump's advantage across 100,000 samples of 1500 voters for the state of Pennsylvania and store the results of each simulation in a list called `simulations`. 

That is, `simulations[i]` should be Trump's proportion advantage for the `i+1`th simple random sample.

<!--
BEGIN QUESTION
name: q6c
points: 1
-->

In [51]:
simulations = ...

In [ ]:
grader.check("q6c")

<!-- BEGIN QUESTION -->

#### Part 4

Make a histogram of the sampling distribution of Trump's proportion advantage in Pennsylvania. Make sure to give your plot a title and add labels where appropriate.
Hint: You should use the [`plt.hist`](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.hist.html) function in your code.

Make sure to include a title as well as axis labels. You can do this using `plt.title`, `plt.xlabel`, and `plt.ylabel`.

<!--
BEGIN QUESTION
name: q6d
manual: true
points: 1
-->

In [53]:
...

<!-- END QUESTION -->

#### Part 5

Now write a function `trump_wins(N)` that creates a sample of N voters for each of the four crucial states (Florida, Michigan, Pennsylvania, and Wisconsin) and returns 1 if Trump is predicted to win based on these samples and 0 if Trump is predicted to lose.

Recall that for Trump to win the election, he must either:
* Win the state of Florida and 1 or more other states
* Win Michigan, Pennsylvania, and Wisconsin

<!--
BEGIN QUESTION
name: q6e
manual: false
points: 2
-->

In [54]:
def trump_wins(N):
    ...

In [ ]:
grader.check("q6e")

#### Part 6

If we repeat 100,000 simulations of the election, i.e. we call `trump_wins(1500)` 100,000 times, what proportion of these simulations predict a Trump victory? Give your answer as `proportion_trump`.

This number represents the percent chance that a given sample will correctly predict Trump's victory *even if the sample was collected with absolutely no bias*. 

**Note: Many laypeople, even well educated ones, assume that this number should be 1. After all, how could a non-biased sample be wrong? This is the type of incredibly important intuition we hope to develop in you throughout this class and your future data science coursework.**

<!--
BEGIN QUESTION
name: q6f
manual: false
points: 1
-->

In [56]:
proportion_trump = ...
proportion_trump

In [ ]:
grader.check("q6f")

We have just studied the sampling error, and found how 
our predictions might look if there was no bias in our 
sampling process. 
Essentially, we assumed that the people surveyed didn't change their minds, 
didn't hide who they voted for, and were representative
of those who voted on election day.

---
## Simulation Study of Selection Bias

According to [an article](https://blogs.lse.ac.uk/usappblog/2018/02/01/better-poll-sampling-would-have-cast-more-doubt-on-the-potential-for-hillary-clinton-to-win-the-2016-election/#Author) by Grotenhuis, Subramanian, Nieuwenhuis, Pelzer and Eisinga:

> In a perfect world, polls sample from the population of voters, who would state their political preference perfectly clearly and then vote accordingly.

That's the simulation study that we just performed. 


It's difficult to control for every source of selection bias.
And, it's not possible to control for some of the other sources of bias.

Next we investigate the effect of small sampling bias on the polling results in these four battleground states.  

Throughout this problem, we'll examine the impacts of a 0.5 percent bias in favor of Clinton in each state. Such a bias has been suggested because highly educated voters tend to be more willing to participate in polls.

### Question 7

Throughout this problem, adjust the selection of voters so that there is a 0.5% bias in favor of Clinton in each of these states. 

For example, in Pennsylvania, Clinton received 47.46 percent of the votes and Trump 48.18 percent. Increase the population of Clinton voters to 47.46 + 0.5  percent and correspondingly decrease the percent of Trump voters. 

#### Part 1

Simulate Trump's advantage across 100,000 simple random samples of 1500 voters for the state of Pennsylvania and store the results of each simulation in a list called `biased_simulations`.

That is, `biased_simulations[i]` should hold the result of the `i+1`th simulation.

That is, your answer to this problem should be just like your answer from Question 6 part 3, but now using samples that are biased as described above.

<!--
BEGIN QUESTION
name: q7a
points: 1
-->

In [59]:
def draw_biased_state_sample(N, state):
    
    ...
    
biased_simulations = ...

In [ ]:
grader.check("q7a")

<!-- BEGIN QUESTION -->

#### Part 2

Make a histogram of the new sampling distribution of Trump's proportion advantage now using these biased samples. That is, your histogram should be the same as in Q6.4, but now using the biased samples.

Make sure to give your plot a title and add labels where appropriate.


<!--
BEGIN QUESTION
name: q7b
manual: true
points: 1
-->

In [65]:
...

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

#### Part 3

Note some similarities and differences between the histogram you created in Q7.2 to that in Q6.4. 

**Hint: Although it is not required, you may find that plotting both histograms on top of each other will help in your analysis.**

<!--
BEGIN QUESTION
name: q7c
manual: true
points: 2
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

#### Part 4

Now perform 100,000 simulations of all four states and return the proportion of these simulations that result in a Trump victory. This is the same fraction that you computed in Question 6 Part 6, but now using your biased samples.

Give your answer as `proportion_trump_biased`.

This number represents the chance that a sample biased 0.5% in Hillary Clinton's favor will correctly predict Trump's victory. You should observe that the chance is signficantly lower than with an unbiased sample, i.e. your answer in Q6.6.

<!--
BEGIN QUESTION
name: q7d
manual: false
points: 1
-->

In [66]:
def trump_wins_biased(N): 
    ...

proportion_trump_biased = ...
proportion_trump_biased

In [ ]:
grader.check("q7d")

## Further Study


### Question 8

Would increasing the sample size have helped?

#### Part 1

Try a sample size of 5,000 and run 100,000 simulations of a sample with replacement. What proportion of the 100,000 times is Trump predicted to win the election in the unbiased setting? In the biased setting?

Give your answers as `high_sample_size_unbiased_proportion_trump` and `high_sample_size_biased_proportion_trump`.

*Note: Don't worry if your solution takes a while to run.* 

<!--
BEGIN QUESTION
name: q8a
manual: false
points: 1
-->



In [69]:
high_sample_size_unbiased_proportion_trump = ...
high_sample_size_biased_proportion_trump = ...
print(high_sample_size_unbiased_proportion_trump, high_sample_size_biased_proportion_trump)

In [ ]:
grader.check("q8a")

#### Part 2

What do your observations from Part 1 say about the impact of sample size
on the sampling error and on the bias?   

Extra question for those who are curious: Just for fun, you might find it interesting to see what happens with even larger sample sizes (> 5000 voters) for both the unbiased and biased cases. Can you get them up to 99% success with sufficient large samples? How many? Why or why not? If you do this, include your observations in your answer.

In [72]:
# Feel free to use this cell for any scratch work (creating visualizations, examining data, etc.)

<!-- BEGIN QUESTION -->

Write your answer in the cell below.

<!--
BEGIN QUESTION
name: q8b
manual: true
points: 2
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 9

According to FiveThirtyEight: "... Polls of the November 2016 presidential election were about as accurate as polls of presidential elections have been on average since 1972."

When the margin of victory may be relatively small as it was in 2016, why don't polling agencies simply gather significantly larger samples to bring this error close to zero? Consider whether or not drawing larger samples will reduce the particular bias inherent in our problem.  

<!--
BEGIN QUESTION
name: q9
manual: true
points: 2
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



**Important**: To make sure the test cases run correctly, click `Kernel>Restart & Run All` and make sure all of the test cases are still passing.

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()